In [1]:
import pandas as pd
import numpy as np
import math
import os
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
import Powerset as ps
from gensim import corpora, models, similarities
import os
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import time
import warnings
import random
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")

df['industry'] = df['ggroup']
df['quarter_year'] = df["year"].astype("str") + " q" + df["quarter"].astype("str")

df2 = df.dropna(subset=['industry']).reset_index(drop=True)
df2['industry-quarter'] = list(zip(df2.industry, df2.quarter_year))
list_industries_quarter = df2.groupby('industry-quarter').count().reset_index()['industry-quarter'].tolist()

In [80]:
df2.count()['Available'] # Total number of reports

86867

In [81]:
temp=df2.groupby(['TICKER','Company Name']).count()['Available'].reset_index()
temp2=temp.groupby(['TICKER']).apply(lambda x: x.sort_values(['Available'], ascending=False))
del temp2['TICKER']
temp2=temp2.reset_index()
temp3=temp2.groupby('TICKER').first().reset_index()

In [82]:
temp3

,TICKER,level_1,Company Name,Available
0,AAL,9,American Airlines Group Inc,221
1,AAP,35,Advance Auto Parts Inc,190
2,AAPL,65,Apple Inc,484
3,ABBV,258,Abbvie Inc,254
4,ABC,345,AmerisourceBergen Corp,181
...,...,...,...,...
425,XRX,6950,Xerox Holdings Corp,35
426,YUM,6962,Yum! Brands Inc,163
427,ZBH,6969,Zimmer Biomet Holdings Inc,208
428,ZBRA,6973,Zebra Technologies Corp,125


In [58]:
temp3['Available'].sum() # Total number of reports

81492

In [62]:
temp3['TICKER_Comp']=temp3['TICKER']+'_'+temp3['Company Name']
list_sample=temp3['TICKER_Comp'].tolist()

In [77]:
df2['TICKER_Comp']=df2['TICKER']+'_'+df2['Company Name']

In [78]:
# select only relevant rows
df3=df2[df2['TICKER_Comp'].apply(lambda x: x in list_sample)]

In [79]:
df3.count()

Available                71685
Date                     71685
Doc Grp                  71685
Info                     71685
Prim. Ticker             71321
Second. Ticker            5266
RI                       69017
Title                    71685
Pages                    71675
Price                    71683
Contributor              71685
Analyst                  67883
Estimate Rating          45025
Recommendation Rating    36397
Company Name             71685
Country                  71645
Industry                 71682
Relevance                    0
DCN                      71685
TICKER                   71685
PrimaryTicker            71685
SecondaryTicker          71685
Match_Ticker             71685
GenderAnalyst            67883
year                     71685
quarter                  71685
PERMNO                   71685
gsector                  71685
ggroup                   71685
industry                 71685
quarter_year             71685
industry-quarter         71685
TICKER_C

In [94]:
df[df['industry'].isna()].groupby(['TICKER','Company Name']).count().to_csv('NonMatched_Industry.csv')

In [86]:
df

,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,Price,...,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,industry,quarter_year,TICKER_Comp
0,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Increase, TargetPrice Increase","3Q18 Recap: Maintaining Our Rating, Raising Ou...",9.0,92.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,AAL_American Airlines Group Inc
1,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,Morningstar | Raising Our FVE After American S...,13.0,69.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,AAL_American Airlines Group Inc
2,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, TargetPrice Increase",Morningstar | Rising Fuel Costs Raise American...,25.0,195.5,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,AAL_American Airlines Group Inc
3,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,3Q18 Recap; Strategic Initiatives Aimed At Dri...,5.0,80.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,AAL_American Airlines Group Inc
4,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, Recommendation Decrease, Ta...",American Airlines Group Inc.,9.0,69.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,AAL_American Airlines Group Inc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98729,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation - 4Q Core EPS $0.80 - So...,12.0,92.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,ZION_Zions Bancorporation NA
98730,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Solid Fees Offset Slightly Smaller Balance She...,7.0,100.0,...,1,NaN,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,ZION_Zions Bancorporation NA
98731,2018-02-05,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,4Q17 - Post Conference Call Model Update,7.0,69.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,ZION_Zions Bancorporation NA
98732,2018-01-17,2018-01-10,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation,9.0,69.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,ZION_Zions Bancorporation NA
